Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.  
Copyright (c) 2017, Pytorch contributors All rights reserved.
## BSD 3-Clause License
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# Model Migration from GPU to the Intel&reg; Gaudi&reg; 2 AI Processor 

The GPU Migration toolkit simplifies migrating PyTorch models that run on GPU-based architecture to run on the Intel® Gaudi® AI accelerator. Rather than manually replacing Python API calls that have dependencies on GPU libraries with Gaudi-specific API calls, the toolkit automates this process so you can run your model with fewer modifications.  

In this notebook we will demonstrate how to use the GPU Migration toolset on a ResNet50 model which is based on open source implementation of ResNet50.  

Refer to the [GPU Migration Toolkit](https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.html) for more information.  

In addtion to this ResNet50 migration, there are addtional GPU Migration example on the Intel Gaudi GitHub page [here](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/examples/gpu_migration)

#### Clone the Model-References repository

In [1]:
%cd ~/Gaudi-tutorials/PyTorch/GPU_Migration
!git clone -b 1.17.1 https://github.com/habanaai/Model-References

/root/Gaudi-tutorials/PyTorch/GPU_Migration
Cloning into 'Model-References'...


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


remote: Enumerating objects: 18994, done.
remote: Counting objects: 100% (4783/4783), done.
remote: Compressing objects: 100% (2090/2090), done.
remote: Total 18994 (delta 2579), reused 4521 (delta 2379), pack-reused 14211 (from 1)
Receiving objects: 100% (18994/18994), 119.29 MiB | 47.73 MiB/s, done.
Resolving deltas: 100% (10265/10265), done.
Updating files: 100% (1440/1440), done.


#### Download dataset (Optional)
To fully run this example you can download [Tiny ImageNet dataset](http://cs231n.stanford.edu/tiny-imagenet-200.zip). It needs to be organized according to PyTorch requirements, and as specified in the scripts of [imagenet-multiGPU.torch](https://github.com/soumith/imagenet-multiGPU.torch).   
Run the cell below to continue downloading the dataset to your local folder and set it up for training.

> **Note: You do not need to have the dataset loaded to see the Migration steps and logging.**

In [2]:
!wget --progress=bar:force http://cs231n.stanford.edu/tiny-imagenet-200.zip
!chmod 600 ./tiny-imagenet-200.zip
import os;os.makedirs("/root/datasets/", exist_ok=True)
!unzip -q ./tiny-imagenet-200.zip  -x "tiny-imagenet-200/test*" -d /root/datasets/

--2024-09-10 20:25:51--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving proxy-us.intel.com (proxy-us.intel.com)... 10.7.211.16
Connecting to proxy-us.intel.com (proxy-us.intel.com)|10.7.211.16|:911... connected.
Proxy request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2024-09-10 20:25:51--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to proxy-us.intel.com (proxy-us.intel.com)|10.7.211.16|:912... connected.
Proxy request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  78.0MB/s    in 3.0s    

2024-09-10 20:25:54 (78.0 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



#### Navigate to the model example to begin the run

In [3]:
%cd ~/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision

/root/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Import Habana Torch Library
Look into train.py, you will see Line 20 where we import Intel Gaudi Torch core library for enabling lazy mode: 

In [ ]:
%%sh
cat -n train.py | head -n 20 | tail -n 2

    19	from torchvision.transforms.functional import InterpolationMode
    20	import habana_frameworks.torch.core as htcore


#### Placing mark_step()
For lazy mode runs, you will have to place the mark_step() function after the optimizer and loss.backward calculations.

In [ ]:
%%sh
cat -n train.py | head -n 50 | tail -n 14

    37	        optimizer.zero_grad(set_to_none=True)
    38	        if scaler is not None:
    39	            scaler.scale(loss).backward()
    40	            htcore.mark_step()
    41	            if args.clip_grad_norm is not None:
    42	                # we should unscale the gradients of optimizer's assigned params if do gradient clipping
    43	                scaler.unscale_(optimizer)
    44	                nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
    45	            scaler.step(optimizer)
    46	            htcore.mark_step()
    47	            scaler.update()
    48	        else:
    49	            loss.backward()
    50	            htcore.mark_step()


> **Note: Starting from version 1.17.0, The GPU code can now be migrated by setting a single environment variable:
>```bash
>PT_HPU_GPU_MIGRATION=1
>```

#### Run the following command to start multi-HPU training.
We're now ready to run the training.  You will see that we've added GPU migration enabling environment variable `PT_HPU_GPU_MIGRATION=1` at the beginning of the run and using the logging command `GPU_MIGRATION_LOG_LEVEL=1` to show the output. There are no other changes to the run command are needed. Once the training run has started, you will see the log files show exactly where the code changes are happening to change from GPU to Intel Gaudi, including the file name and location.

```bash
PT_HPU_GPU_MIGRATION=1 GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path="/root/datasets/tiny-imagenet-200/" --workers=8 --epochs=1 --opt=sgd --amp
```

In [4]:
!PT_HPU_GPU_MIGRATION=1 GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path="/root/datasets/tiny-imagenet-200/" --workers=8 --epochs=1 --opt=sgd --amp

/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/gpu_migration/__init__.py:46: UserWarning: apex not installed, gpu_migration will not swap api for this package.
  warnings.warn(
gpu migration log will be saved to /var/log/habana_logs/gpu_migration_logs/2024-09-10/21-08-02/gpu_migration_3079.log
[2024-09-10 21:08:02] /root/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision/utils.py:265
    [context]:     torch.cuda.set_device(args.gpu)

    [hpu_match]: torch.cuda.set_device(device=0, ) --> torch.hpu.set_device(hpu:0)

| distributed init (rank 0): env://
[2024-09-10 21:08:02] /root/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification

In [5]:
exit()